# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../WeatherPy/output_data/weather_data.csv')
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature °F,Humidity (%),Cloud Coverage (%),Wind Speed (MPH)
0,0,Paamiut,59.702501,-52.265104,25.92,86.0,60.0,8.23
1,1,Bandarbeyla,6.657520,58.129427,73.60,74.0,85.0,13.27
2,2,Nizhneyansk,89.893231,133.970288,NaN,NaN,NaN,NaN
3,3,Rikitea,-34.638351,-113.096852,79.09,71.0,82.0,8.52
4,4,Mehamn,78.996117,27.497730,26.60,86.0,0.0,12.66


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(weather_df[['Latitude', 'Longitude']], weights= weather_df['Humidity (%)'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
ideal_df = weather_df.loc[(weather_df['Temperature \N{DEGREE SIGN}F'] > 70) & 
                          (weather_df['Temperature \N{DEGREE SIGN}F'] < 90) &
                          (weather_df['Humidity (%)'] < 90) &
                          (weather_df['Cloud Coverage (%)'] == 0)
                         ]
print(ideal_df.shape)
ideal_df

(21, 8)


,Unnamed: 0,City,Latitude,Longitude,Temperature °F,Humidity (%),Cloud Coverage (%),Wind Speed (MPH)
25,25,Nelson Bay,-35.424303,156.864870,70.11,73.0,0.0,6.91
35,35,Saint-Philippe,-49.718899,67.090028,76.41,65.0,0.0,5.75
46,46,Puerto Colombia,13.198561,-76.233720,82.40,69.0,0.0,17.27
100,100,Atuona,-13.550906,-141.090973,79.23,73.0,0.0,12.12
129,129,Pisco,-20.463752,-86.593766,77.20,69.0,0.0,5.75
159,159,Tchamba,9.123938,1.508974,82.40,36.0,0.0,0.67
162,162,Viedma,-47.876891,-57.114958,84.00,39.0,0.0,1.99
175,175,Aquiraz,-2.206836,-36.064720,82.40,74.0,0.0,13.80
183,183,Neuquen,-38.015997,-69.181892,89.60,35.0,0.0,4.61
215,215,Conde,-15.046420,-32.595910,80.60,74.0,0.0,12.66


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = ideal_df
hotel_df['Hotel Name'] = ""
hotel_df['Country'] = ""
hotel_df = hotel_df.reset_index()
hotel_df.head()

C:\Users\Owner\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Owner\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,index,Unnamed: 0,City,Latitude,Longitude,Temperature °F,Humidity (%),Cloud Coverage (%),Wind Speed (MPH),Hotel Name,Country
0,25,25,Nelson Bay,-35.424303,156.864870,70.11,73.0,0.0,6.91,,
1,35,35,Saint-Philippe,-49.718899,67.090028,76.41,65.0,0.0,5.75,,
2,46,46,Puerto Colombia,13.198561,-76.233720,82.40,69.0,0.0,17.27,,
3,100,100,Atuona,-13.550906,-141.090973,79.23,73.0,0.0,12.12,,
4,129,129,Pisco,-20.463752,-86.593766,77.20,69.0,0.0,5.75,,


In [38]:
# geocoordinates
target_coordinates = ""
target_radius = 10000 * 1000
target_type = "lodging"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotels = []
countries = []

for index, row in hotel_df.iterrows():
    
    target_coordinates = f'{row["Latitude"]},{row["Longitude"]}'

    # set up a parameters dictionary
    params = {
    "location": target_coordinates,    
    "type": target_type,   
    "radius": target_radius,
    "key": gkey   
    }

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()

    try:
        hotel_name = response['results'][0]['name']
        country_response = response['results'][0]['plus_code']['compound_code']
        country_name = country_response.split(", ",1)[1]
        
        hotels.append(hotel_name)
        countries.append(country_name)
        
    except:
        
        print(f'No results for #{index}: {row["City"]}. Skipping...')
        hotels.append("")
        countries.append("")
        

No results for #0: Nelson Bay. Skipping...
No results for #1: Saint-Philippe. Skipping...
No results for #3: Atuona. Skipping...
No results for #4: Pisco. Skipping...
No results for #5: Tchamba. Skipping...
No results for #6: Viedma. Skipping...
No results for #7: Aquiraz. Skipping...
No results for #8: Neuquen. Skipping...
No results for #9: Conde. Skipping...
No results for #10: Saint-Joseph. Skipping...
No results for #13: Faanui. Skipping...
No results for #15: Noumea. Skipping...
No results for #19: Praia. Skipping...
No results for #20: Tevaitoa. Skipping...


In [44]:
hotel_df['Hotel Name'] = hotels
hotel_df['Country'] = countries
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != ""]
hotel_df

,index,Unnamed: 0,City,Latitude,Longitude,Temperature °F,Humidity (%),Cloud Coverage (%),Wind Speed (MPH),Hotel Name,Country
2,46,46,Puerto Colombia,13.198561,-76.233720,82.40,69.0,0.0,17.27,Garden Park Resort,Ghana
11,286,286,Mecca,22.644109,42.529666,75.31,58.0,0.0,2.64,Ultimate Guest House,Ghana
12,313,313,Bara,16.219800,28.560012,73.49,19.0,0.0,7.74,Hotel Dynasty,Ghana
14,340,340,Salalah,13.716721,60.649697,71.60,53.0,0.0,3.44,Mountain Paradise Lodge,Ghana
16,413,413,Tessalit,20.157291,3.054955,70.86,17.0,0.0,8.66,Ropads Guest House,Ghana
17,441,441,Chabahar,24.146920,60.242519,70.34,46.0,0.0,5.64,Auberge de la Belle Etoile,Burkina Faso
18,458,458,Nouadhibou,22.950674,-14.917286,71.60,49.0,0.0,18.41,Résidence le Cotonnier,Burkina Faso


In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

,Latitude,Longitude
2,13.198561,-76.233720
11,22.644109,42.529666
12,16.219800,28.560012
14,13.716721,60.649697
16,20.157291,3.054955
17,24.146920,60.242519
18,22.950674,-14.917286


In [48]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)


fig.add_layer(hotel_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))